In [3]:
#import all necessary items from Su's code:
#from xd import *
#from utils import *
import esutil, yaml, sys, os, argparse
import healpy as hp
#from systematics import *
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import numpy as np
import astropy.io.fits as fits

#opening CMASS catalog:
fitsfile='galaxy_DR11v1_CMASS_South-photoObj.fits.gz'
hdu=fits.open(fitsfile)

import h5py

#opening DES catalog:
filename='/fs/scratch/PCON0003/warner785/Y3_mastercat_03_31_20.h5'
#f=h5py.File(filename,'r')
with h5py.File(filename, 'r') as f:
#read in values:
    modelmag_g_des=f['catalog/gold/sof_cm_mag_corrected_g'][:]
    modelmag_r_des=f['catalog/gold/sof_cm_mag_corrected_r'][:]
    modelmag_i_des=f['catalog/gold/sof_cm_mag_corrected_i'][:]
    
    magerr_detmodel_g=f['catalog/gold/sof_cm_mag_err_g']
    magerr_detmodel_r=f['catalog/gold/sof_cm_mag_err_r']
    magerr_detmodel_i=f['catalog/gold/sof_cm_mag_err_i']
    
    ra=f['catalog/gold/ra'][:]
    dec=f['catalog/gold/dec'][:]
    
cmodelmag_g_des=modelmag_g_des
cmodelmag_r_des=modelmag_r_des
cmodelmag_i_des=modelmag_i_des
magauto_des=modelmag_i_des

print(len(modelmag_g_des))
print(len(ra))
    
#defining the cut:

cut = (((cmodelmag_r_des>16)&(cmodelmag_r_des<24))&((cmodelmag_i_des>16)&(cmodelmag_i_des<24))&((cmodelmag_g_des>16)&(cmodelmag_g_des<24))&((modelmag_r_des-modelmag_i_des)<1.5)&((modelmag_r_des-modelmag_i_des)>0.)&((modelmag_g_des-modelmag_r_des)>0.)&((modelmag_g_des-modelmag_r_des)<2.5)&(magauto_des<21.5))
ra=ra[cut]
dec=dec[cut]

print(len(ra))
#print(modelmag_g_des.size)

print('files loaded successfully')

399263026
399263026
72400610
files loaded successfully


In [2]:
#matching catalogs (train_st82): UNNEEDED STEP
'''
#Su's flags and color cut:
mask_all=cut
f'=f[cut]
train_sample=fitsfile
#print('total num of train', len(train_sample))
'''
#reading in needed values:
cmass_ra=hdu[1].data['RA     ']
cmass_dec=hdu[1].data['DEC    ']

mg1, mg2,_= esutil.htm.HTM(10).match(cmass_ra, cmass_dec, ra, dec, 2./3600, maxmatch=1)

cmass_mask=np.zeros(ra.size, dtype=bool)
cmass_mask[mg2]=1
clean_cmass_data_des_ra, nocmass_ra=ra[cmass_mask], ra[~cmass_mask]
clean_cmass_data_des_dec, nocmass_dec=dec[cmass_mask], dec[~cmass_mask]

print(('num of cmass in des side', clean_cmass_data_des_ra.size, '({:0.0f}%)'.format(clean_cmass_data_des_ra.size*1./cmass_ra.size*100.)))

('num of cmass in des side', 56335, '(30%)')


In [4]:
#match needed values:

#reading in needed cmass values:
cmass_ra=hdu[1].data['RA     ']
cmass_dec=hdu[1].data['DEC    ']

#ra and dec are already read in

from astropy.coordinates import SkyCoord
from astropy import units as u
cmass = SkyCoord(ra=cmass_ra*u.degree, dec=cmass_dec*u.degree)
gold = SkyCoord(ra=ra*u.degree, dec=dec*u.degree)
idx, d2d, d3d = gold.match_to_catalog_sky(cmass)

In [5]:
#matched catalogs:

cmass_matches = cmass[idx]
print(len(cmass_matches))
max_sep = 1.0 * u.arcsec
idx, d2d, d3d = gold.match_to_catalog_3d(cmass)
sep_constraint = d2d < max_sep
gold_matches = gold[sep_constraint]
print(len(gold_matches))
cmass_matches = cmass[idx[sep_constraint]]
print(len(cmass_matches))

72400610
56244
56244


In [6]:
#Creating of Figure 1:

#Read in data:
from cmass_modules import io, Cuts
#cmass_spec = esutil.io.read('galaxy_DR11v1_CMASS_South-photoObj.fits.gz')
cmass_photo = esutil.io.read('galaxy_DR11v1_CMASS_South-photoObj.fits.gz')
cmass_stripe=cmass_photo
##cmass_stripe = Cuts.keepGoodRegion(cmass_photo)
#gold_train, cmass_train = matchCatalogsbyPosition( gold , cmass_stripe)
gold_train=gold_matches
cmass_train=cmass_matches
#gold_train = train_sample.copy()

#BOSS values:
magmodel=hdu[1].data['CMODELMAG']
cmodelmag_g_des=magmodel[:,1]
cmodelmag_r_des=magmodel[:,2]
cmodelmag_i_des=magmodel[:,3]

ext=hdu[1].data['EXTINCTION']
extg=ext[:,1]
extr=ext[:,2]
exti=ext[:,3]

magmodelerr=hdu[1].data['CMODELMAGERR']
cmodelmag_g_err=magmodelerr[:,1]
cmodelmag_r_err=magmodelerr[:,2]
cmodelmag_i_err=magmodelerr[:,3]

cmass_g=cmodelmag_g_des[idx[sep_constraint]]
cmass_r=cmodelmag_r_des[idx[sep_constraint]]
cmass_i=cmodelmag_i_des[idx[sep_constraint]]

cmass_extg=extg[idx[sep_constraint]]
cmass_extr=extr[idx[sep_constraint]]
cmass_exti=exti[idx[sep_constraint]]

cmass_gerr=cmodelmag_g_err[idx[sep_constraint]]
cmass_rerr=cmodelmag_r_err[idx[sep_constraint]]
cmass_ierr=cmodelmag_i_err[idx[sep_constraint]]

print(len(cmass_ierr))

56244


In [ ]:
#SDSS color for CMASS in overlap region
g_sdss = cmass_g-cmass_extg
r_sdss = cmass_r-cmass_extr
i_sdss = cmass_i-cmass_exti
gr_sdss = g_sdss - r_sdss
ri_sdss = r_sdss - i_sdss

print(len(modelmag_g_des))
print(len(ra))

modelmag_g_des=modelmag_g_des[cut]
modelmag_r_des=modelmag_r_des[cut]
modelmag_i_des=modelmag_i_des[cut]
    
magerr_detmodel_g=magerr_detmodel_g[cut]
magerr_detmodel_r=magerr_detmodel_r[cut]
magerr_detmodel_i=magerr_detmodel_i[cut]

print(len(modelmag_g_des))

#CMASS in GOLD catalog in overlap region
gr_des = cmass_g- cmass_r
ri_des = cmass_r- cmass_i

#GOLD catalog in overlap region
gr_gold = modelmag_g_des[sep_constraint]- modelmag_r_des[sep_constraint]
ri_gold = modelmag_r_des[sep_constraint]- modelmag_i_des[sep_constraint]

399263026
72400610


In [ ]:
x = np.linspace(-2,4,10)
dperp = 0.55 + x/8.

gr_sdss_err = np.sqrt(cmass_gerr**2 + cmass_rerr**2)
ri_sdss_err = np.sqrt(cmass_rerr**2 + cmass_ierr**2)

gr_des_err = np.sqrt(magerr_detmodel_g[sep_constraint]**2 +  magerr_detmodel_r[sep_constraint]**2 )
ri_des_err = np.sqrt(magerr_detmodel_r[sep_constraint]**2 +  magerr_detmodel_i[sep_constraint]**2 )

In [ ]:
dperpcut = ((ri_sdss - gr_sdss/8. < 0.55) & 
((gr_sdss>0.5)&(gr_sdss<0.96)|
 ((gr_sdss>1.43)&(gr_sdss<1.435))|
 ((gr_sdss>1.85)&(gr_sdss<3.0)) ) )

dperpcut_des = (ri_des - gr_des/8. < 0.5505) & (ri_des - gr_des/8. > 0.55035)
print(np.sum(dperpcut), np.sum(dperpcut_des))

In [ ]:
#smaller amount of data:

size=gr_gold.size
print(size)
####seed choosing random number
#arrays always the same for each

rand_arr=np.random.random(gr_gold.size)
#sorting this array:
gr_gold2=gr_gold[rand_arr.argsort()]
ri_gold2=ri_gold[rand_arr.argsort()]
gr_sdss2=gr_sdss[rand_arr.argsort()]
ri_sdss2=ri_sdss[rand_arr.argsort()]
gr_des2=gr_des[rand_arr.argsort()]
ri_des2=ri_des[rand_arr.argsort()]
#matching: randomize in the same way, providing the same indices. 

#shorten the randomly selected data to plottable size:
short_gr_gold= gr_gold2[:1000]
short_ri_gold= ri_gold2[:1000]
short_gr_sdss= gr_sdss2[:1000]
short_ri_sdss= ri_sdss2[:1000]
short_gr_des= gr_des2[:1000]
short_ri_des= ri_des2[:1000]

size2=short_gr_gold.size
print(size2)

In [ ]:
#use plot command from matplotlib first: color color plot
#g-r :x etc. 

import matplotlib.pyplot as plt

data_gold = np.column_stack( [short_gr_gold, short_ri_gold] )
data_sdss = np.column_stack( [short_gr_sdss, short_ri_sdss] )
data_des = np.column_stack( [short_gr_des, short_ri_des] )

plt.plot(short_gr_des, short_ri_des, label='CMASS in GOLD catalog in overlap region')
#plt.plot(x, dperp, 'r-', linewidth=1)

#plt.xlabel('g-r')
#plt.ylabel('r-i')
#plt.text(2.0, 0.1, 'DES  Color')
#plt.tight_layout()
plt.show()

In [ ]:
#test:
import matplotlib
#print(matplotlib.rcParams['backend'])

plt.plot(range(10))
plt.show()

In [ ]:
#Now creating first part of Figure 1: DES plot first

from chainconsumer import ChainConsumer

#seed(0)
#cov = normal(size=(3, 3))
#data = multivariate_normal(normal(size=3), np.dot(cov, cov.T), size=100000)

data_gold = np.column_stack( [gr_gold, ri_gold] )
data_sdss = np.column_stack( [gr_sdss, ri_sdss] )
data_des = np.column_stack( [gr_des, ri_des] )

c = ChainConsumer()
c.add_chain(data_gold, parameters=["$g-r$", "$r-i$"])
#c.add_chain(data_des)
c.configure(plot_hists=False, cloud=True, sigmas=np.linspace(0, 3, 10), colors ='grey', 
            shade_alpha = 0.7, kde=True)


fig = c.plotter.plot()
ax = fig.axes[0]
ax.plot(gr_des, ri_des, 'g.', markersize = 1, alpha = 0.7)
#ax.plot(gr_sdss, ri_sdss, '.',markersize = 1, alpha = 0.7)
ax.plot(x, dperp, 'r-', linewidth=1)
ax.set_ylim(0,2)
ax.set_xlim(0, 3)

ax.text(2.0, 0.1, 'DES  Color')
plt.tight_layout()